In [86]:
import requests
import csv
import os
import pandas as pd
import json
import re
from datetime import datetime
today_string = datetime.now().strftime('%Y-%m-%d')

# Get data from wiag

```sql
SELECT i.id_public, u.value
FROM `item_corpus` i 
INNER JOIN (
    select item_id, value FROM url_external WHERE authority_id=200
) u on i.item_id = u.item_id
INNER JOIN item ON item.id = i.item_id
where item.merge_status = 'parent'
or item.merge_status = 'original'
```

In [87]:
ic_df = pd.read_csv('item_corpus_2024-05-15_1.csv', names=["wiag_id","pd_id"])
ic_df

,wiag_id,pd_id
0,WIAG-Pers-EPISCGatz-04130-001,003-00788-001
1,WIAG-Pers-EPISCGatz-04125-001,003-00686-001
2,WIAG-Pers-EPISCGatz-21161-001,003-00611-001
3,WIAG-Pers-EPISCGatz-21136-001,003-00382-001
4,WIAG-Pers-EPISCGatz-02835-001,003-00368-001
...,...,...
17510,WIAG-Pers-EPISCGatz-03416-001,060-02185-001
17511,WIAG-Pers-EPISCGatz-02576-001,060-02180-001
17512,WIAG-Pers-CANON-49000-001,006-00114-001
17513,WIAG-Pers-CANON-15152-001,076-00269-001


In [88]:
ic_df[ic_df['pd_id'].isna()]

,wiag_id,pd_id


# Get data from personendatenbank

```sql
SELECT persons.wiag, MIN(gsn.nummer)
FROM items
INNER JOIN persons ON persons.item_id = items.id AND persons.deleted=0 AND items.deleted=0 AND items.status = "online"
INNER JOIN gsn ON gsn.item_id = items.id AND gsn.deleted=0
WHERE persons.wiag IS NOT NULL AND persons.wiag != ''
GROUP BY persons.wiag
```


In [89]:
# need to fix the ebene problem
pr_df = pd.read_csv('pr_2024-05-15.csv', names=["wiag_id", "pd_id"])
pr_df

,wiag_id,pd_id
0,WIAG-Pers-CANON-10014-001,054-02832-001
1,WIAG-Pers-CANON-10032-001,032-02373-001
2,WIAG-Pers-CANON-10047-001,029-03106-001
3,WIAG-Pers-CANON-10054-001,029-03206-001
4,WIAG-Pers-CANON-10071-001,029-01071-001
...,...,...
11003,WIAG-Pers-EPISCGatz-21735-001,061-06324-001
11004,WIAG-Pers-EPISCGatz-21750-001,052-02662-001
11005,WIAG-Pers-EPISCGatz-21751-001,077-00515-001
11006,WIAG-Pers-EPISCGatz-21766-001,073-01566-001


In [90]:
pr_df[pr_df['pd_id'].isna()]

,wiag_id,pd_id


In [91]:
pr_df_gp = pr_df.groupby('wiag_id').count()
pr_df_gp[pr_df_gp['pd_id'] > 1]

,pd_id
wiag_id,


In [92]:
joined_df = ic_df.merge(pr_df, on='pd_id', suffixes=('_wiag', '_pd'))
joined_df

,wiag_id_wiag,pd_id,wiag_id_pd
0,WIAG-Pers-EPISCGatz-04130-001,003-00788-001,WIAG-Pers-EPISCGatz-04130-001
1,WIAG-Pers-EPISCGatz-04125-001,003-00686-001,WIAG-Pers-EPISCGatz-04125-001
2,WIAG-Pers-EPISCGatz-21161-001,003-00611-001,WIAG-Pers-EPISCGatz-21161-001
3,WIAG-Pers-EPISCGatz-21136-001,003-00382-001,WIAG-Pers-EPISCGatz-21136-001
4,WIAG-Pers-EPISCGatz-02835-001,003-00368-001,WIAG-Pers-EPISCGatz-02835-001
...,...,...,...
12353,WIAG-Pers-EPISCGatz-05548-001,003-01528-001,WIAG-Pers-EPISCGatz-05548-001
12354,WIAG-Pers-EPISCGatz-03416-001,060-02185-001,WIAG-Pers-EPISCGatz-03416-001
12355,WIAG-Pers-EPISCGatz-02576-001,060-02180-001,WIAG-Pers-EPISCGatz-02576-001
12356,WIAG-Pers-CANON-49000-001,006-00114-001,WIAG-Pers-CANON-84854-001


In [93]:
unequal_df = joined_df[joined_df['wiag_id_wiag'] != joined_df['wiag_id_pd']]
unequal_df

,wiag_id_wiag,pd_id,wiag_id_pd
893,WIAG-Pers-CANON-12751-001,003-02382-001,WIAG-Pers-CANON-80093-001
894,WIAG-Pers-CANON-19151-001,003-00620-001,WIAG-Pers-CANON-19495-001
895,WIAG-Pers-CANON-18654-001,003-00269-001,WIAG-Pers-EPISCGatz-03473-001
896,WIAG-Pers-CANON-12710-001,002-03124-001,WIAG-Pers-CANON-84711-001
904,WIAG-Pers-CANON-25818-001,012-00035-001,WIAG-Pers-CANON-13382-001
...,...,...,...
12346,WIAG-Pers-CANON-52627-001,083-00093-001,WIAG-Pers-CANON-89052-001
12351,WIAG-Pers-CANON-25887-001,052-01515-001,WIAG-Pers-CANON-85102-001
12352,WIAG-Pers-CANON-15116-001,076-01659-001,WIAG-Pers-CANON-85559-001
12356,WIAG-Pers-CANON-49000-001,006-00114-001,WIAG-Pers-CANON-84854-001


In [94]:
# check_list = list(map(lambda x: x[-9:-8] == '8' or x[-9:-8] == '9', unequal_df[unequal_df['wiag_id_x'].isna()]['wiag_id_y'].to_list()))
# any(check_list)
empty_wiag = unequal_df[unequal_df['wiag_id_wiag'].isna()]
diff_df = empty_wiag[empty_wiag.wiag_id_pd.str.contains('[89][0-9]{4}-[0-9]{3}$', regex=True)]
diff_df

,wiag_id_wiag,pd_id,wiag_id_pd


In [95]:
unequal_df = unequal_df[~unequal_df['wiag_id_wiag'].isna()]
unequal_df

,wiag_id_wiag,pd_id,wiag_id_pd
893,WIAG-Pers-CANON-12751-001,003-02382-001,WIAG-Pers-CANON-80093-001
894,WIAG-Pers-CANON-19151-001,003-00620-001,WIAG-Pers-CANON-19495-001
895,WIAG-Pers-CANON-18654-001,003-00269-001,WIAG-Pers-EPISCGatz-03473-001
896,WIAG-Pers-CANON-12710-001,002-03124-001,WIAG-Pers-CANON-84711-001
904,WIAG-Pers-CANON-25818-001,012-00035-001,WIAG-Pers-CANON-13382-001
...,...,...,...
12346,WIAG-Pers-CANON-52627-001,083-00093-001,WIAG-Pers-CANON-89052-001
12351,WIAG-Pers-CANON-25887-001,052-01515-001,WIAG-Pers-CANON-85102-001
12352,WIAG-Pers-CANON-15116-001,076-01659-001,WIAG-Pers-CANON-85559-001
12356,WIAG-Pers-CANON-49000-001,006-00114-001,WIAG-Pers-CANON-84854-001


In [96]:
unequal_df['pd_link'] = unequal_df.apply(lambda row: "http://personendatenbank.germania-sacra.de/index/gsn/" + row["pd_id"], axis = 1)
unequal_df
unequal_df['current_wiag_link'] = unequal_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_wiag"], axis = 1)
unequal_df
unequal_df['pd_wiag_link'] = unequal_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_pd"], axis = 1)
unequal_df

C:\Users\khan32\AppData\Local\Temp\ipykernel_12324\3472669850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unequal_df['pd_link'] = unequal_df.apply(lambda row: "http://personendatenbank.germania-sacra.de/index/gsn/" + row["pd_id"], axis = 1)
C:\Users\khan32\AppData\Local\Temp\ipykernel_12324\3472669850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unequal_df['current_wiag_link'] = unequal_df.apply(lambda row: "https://wiag-vocab.adw-goe.de/id/" + row["wiag_id_wiag"], axis = 1)
C:\Users\khan32\A

,wiag_id_wiag,pd_id,wiag_id_pd,pd_link,current_wiag_link,pd_wiag_link
893,WIAG-Pers-CANON-12751-001,003-02382-001,WIAG-Pers-CANON-80093-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
894,WIAG-Pers-CANON-19151-001,003-00620-001,WIAG-Pers-CANON-19495-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
895,WIAG-Pers-CANON-18654-001,003-00269-001,WIAG-Pers-EPISCGatz-03473-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-EPI...
896,WIAG-Pers-CANON-12710-001,002-03124-001,WIAG-Pers-CANON-84711-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
904,WIAG-Pers-CANON-25818-001,012-00035-001,WIAG-Pers-CANON-13382-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
...,...,...,...,...,...,...
12346,WIAG-Pers-CANON-52627-001,083-00093-001,WIAG-Pers-CANON-89052-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
12351,WIAG-Pers-CANON-25887-001,052-01515-001,WIAG-Pers-CANON-85102-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
12352,WIAG-Pers-CANON-15116-001,076-01659-001,WIAG-Pers-CANON-85559-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...
12356,WIAG-Pers-CANON-49000-001,006-00114-001,WIAG-Pers-CANON-84854-001,http://personendatenbank.germania-sacra.de/ind...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...,https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CAN...


In [97]:
unequal_df[['pd_link', 'current_wiag_link', 'pd_wiag_link']].to_csv(f'unequal_check_{today_string}.csv', sep=';')

In [99]:
### https://wiag-vocab.adw-goe.de/id/WIAG-Pers-CANON-12751-001 doesn't exist on wiag???
### check for fertig status

In [9]:
# SELECT *  FROM url_external WHERE authority_id=200

In [10]:

# select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200)

In [11]:
# select * from person where id in
# (select id_in_source from item 
# where id_in_source is not NULL 
# and id_in_source!=''
# and id in 
# (SELECT item_id FROM url_external WHERE authority_id=200))

In [12]:
# SELECT * FROM `item_corpus` 
# where item_id in  (SELECT item_id FROM url_external WHERE authority_id=200)

In [13]:
# SELECT i.id_public, i.item_id, u.value 
# FROM `item_corpus` i 
# INNER JOIN (
#     select item_id, value FROM url_external WHERE authority_id=200
# ) u on i.item_id = u.item_id

,wiag_id,pd_id
0,WIAG-Pers-EPISCGatz-04130-001,003-00788-001
1,WIAG-Pers-EPISCGatz-04125-001,003-00686-001
2,WIAG-Pers-EPISCGatz-21161-001,003-00611-001
3,WIAG-Pers-EPISCGatz-21136-001,003-00382-001
4,WIAG-Pers-EPISCGatz-02835-001,003-00368-001
...,...,...
19476,WIAG-Pers-CANON-26121-001,006-00156-001
19477,WIAG-Pers-CANON-48819-001,006-00156-001
19478,WIAG-Pers-CANON-26121-001,006-00156-001
19479,WIAG-Pers-CANON-25731-001,054-00165-001


In [ ]:
# example complex query
# http://personendatenbank.germania-sacra.de/api/v1.0/person?
# query[0][field]=person.vorname&
# query[0][value]=b*&
# query[0][operator]=like&
# query[0][connector]=or&
# query[1][field]=person.familienname&
# query[1][value]=b*&
# query[1][operator]=like&
# format=json-ld
